# groupBy After groupBy!

Basically, After groupBy, it can't be groupBy again. <br>
So we should treat this with a little trick! <br>

The thing is after groupBy, we will save the data into the folder.
and then, we reload the data. then, we can expect to do groupBy again.

# Load SparkSession and Data

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col

from pyspark.sql.functions import to_timestamp
from pyspark.sql import types

spark = SparkSession \
    .builder \
    .appName("Window Operations on Event Time") \
    .getOrCreate()

static_df = spark.read.json("./data/sample.txt")

schema = static_df.schema

df = spark.readStream.format("json").schema(schema).option("maxFilesPerTrigger", 1).load("./data/")

print(df)
static_df.show(5, truncate=False)

DataFrame[created_at: string, sentiment_level: string]
+-------------------+---------------+
|created_at         |sentiment_level|
+-------------------+---------------+
|2021-10-29T18:10:09|2              |
+-------------------+---------------+



# Window operation with water mark
### Conditions for watermarking to clean aggregation state <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#quick-example">[link]</a> 

In [2]:
from pyspark.sql.types import TimestampType
tsDF = df.select(col("created_at").cast(TimestampType()), col("sentiment_level"))

In [3]:
# If you set a WaterMark with a time, it blocks not to be data included x hours/minutes/seconds passed.

from pyspark.sql.functions import window

windowedCounts = tsDF \
    .withWatermark("created_at", "10 seconds") \
    .groupBy(
        window(col("created_at"), "10 seconds", "5 seconds").alias("created_at"),
        col("sentiment_level")) \
    .count()

# Save your calculated data into the folder.

In [4]:
windowedCounts \
    .writeStream \
    .format("json") \
    .option("path", "./windowsedCounts_output/") \
    .option("checkpointLocation", "./windowsedCounts_checkpoint/") \
    .queryName("df") \
.start()

In [6]:
windowedCountsDF = spark.readStream.format("json").option("path", "./windowsedCounts_output/").load()

IllegalArgumentException: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.